In [1]:
import torch
import numpy as np
import cv2

import matplotlib.pyplot as plt
import pickle as pkl

import os
import random
import shutil

from PIL import Image
from torchvision.datasets import MNIST

In [2]:
#!pip install opencv-python

In [3]:
plt.rcParams['image.cmap'] = 'Blues_r'

## Save to Training Format

In [7]:
num_samples = 1000
data_dir = './gans_data/data_google_large'

# create directory to save training data
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [8]:
# delete existing data
# shutil.rmtree(data_dir)

In [9]:
# take random images from training data instead

org_data_directory='./data/google_digit_data/train_cropped_32'
test_imgs = os.listdir(org_data_directory+'/images')
test_idx = [i.split('.')[0] for i in test_imgs]

In [10]:
len(test_idx)

5251

In [11]:
random.shuffle(test_idx)
test_idx = test_idx[:num_samples]

In [12]:
# generate and save YOLO data and annotations

image_list = []
classes_list = []
annotations_list = []

for idx in test_idx:
    
    # read image
    img = np.array(Image.open(f"{org_data_directory}/images/{idx}.png"))
    img = np.moveaxis(img, -1, 0)
    
    # read corresponding annotation
    with open(f"{org_data_directory}/labels/{idx}.txt", 'r') as f:
        annotation = f.read()
        annotation = annotation.split('\n')[:-1]
        f.close()

    cls = [int(i[0]) for i in annotation]
    bbox = [i.split(' ')[1:] for i in annotation]
    for idx, val in enumerate(bbox):
        bbox[idx] = [float(val) for val in bbox[idx]]

    image_list.append(torch.tensor(img).double()) # unsqueeze(0).double()
    classes_list.append(torch.tensor(cls).double())
    annotations_list.append(torch.tensor(bbox).double())


In [13]:
print(len(image_list))
print(image_list[0].shape)

1000
torch.Size([3, 32, 32])


In [15]:
# t = np.array(image_list[0].int())
# t = np.moveaxis(np.array(image_list[7].int()), 0, 2)
# plt.imshow(t)

In [14]:
print(len(classes_list))
print(classes_list[0].shape)
print(classes_list[0])

1000
torch.Size([3])
tensor([3., 0., 0.], dtype=torch.float64)


In [15]:
print(len(annotations_list))
print(annotations_list[0].shape)
print(annotations_list[0])

1000
torch.Size([3, 4])
tensor([[0.2969, 0.3906, 0.2812, 0.4062],
        [0.5312, 0.3594, 0.2500, 0.4062],
        [0.8125, 0.3906, 0.3125, 0.4062]], dtype=torch.float64)


In [16]:
with open(f'{data_dir}/image_list', 'wb+') as f:
    pkl.dump(image_list, f)
    
with open(f'{data_dir}/classes_list', 'wb+') as f:
    pkl.dump(classes_list, f)
    
with open(f'{data_dir}/annotations_list', 'wb+') as f:
    pkl.dump(annotations_list, f)